<a href="https://colab.research.google.com/github/cbalkig/GoogleDriveFileDownloader/blob/main/GoogleDriveFileDownloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
SRC_URLS = ["https://download.pytorch.org/whl/cu116/torch-1.13.0%2Bcu116-cp38-cp38-win_amd64.whl",
            "https://download.pytorch.org/whl/cu117/torch-1.13.0%2Bcu117-cp38-cp38-win_amd64.whl",
            "https://download.pytorch.org/whl/cu116/torch-1.13.1%2Bcu116-cp38-cp38-win_amd64.whl",
            "https://download.pytorch.org/whl/cu117/torch-1.13.1%2Bcu117-cp38-cp38-win_amd64.whl",
            "https://developer.download.nvidia.com/compute/cuda/12.0.0/local_installers/cuda_12.0.0_527.41_windows.exe"]

GOOGLE_DRIVE_PATH = "NVIDIA"

In [2]:
from google.colab import drive
drive.mount('/google_drive')

Drive already mounted at /google_drive; to attempt to forcibly remount, call drive.mount("/google_drive", force_remount=True).


In [3]:
import requests
import os
import urllib.parse
import base64
import hashlib
from tqdm import tqdm

In [4]:
main_folder_path = os.path.join("/google_drive", 'MyDrive', GOOGLE_DRIVE_PATH)

In [5]:
def process_file(SRC_URL):
    print(SRC_URL)
    r = requests.get(SRC_URL, stream = True)
    print(r.status_code)

    file_name = urllib.parse.unquote(SRC_URL.rsplit('/', 1)[-1])
    target_file_path = os.path.join(main_folder_path, file_name)
    print(target_file_path)

    if os.path.exists(target_file_path):
        print("File is already present")
        return

    if r.status_code != 200:
        print(r)
        return

    with open(target_file_path, "wb") as file:
        total_size_in_bytes = int(r.headers.get('content-length', 0))
        progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

        m = hashlib.md5()
        for data in r.iter_content(chunk_size = 1024):
            if not data:
                break
            progress_bar.update(len(data))
            file.write(data)
            m.update(data)
        progress_bar.close()

        print("Remote MD5:", m.hexdigest())
    
    local_md5 = get_local_md5(target_file_path)
    print("Local MD5:", local_md5)

    print("File saved to", target_file_path)
    print("\n")

In [6]:
def get_local_md5(file_path):
    with open(file_path, "rb") as file:
        m = hashlib.md5()
        while True:
            data = file.read(1024)
            if not data:
                break
            m.update(data)
        return m.hexdigest()


In [7]:
target_paths = []
for SRC_URL in SRC_URLS:
    process_file(SRC_URL)
    print("\n")

https://download.pytorch.org/whl/cu116/torch-1.13.0%2Bcu116-cp38-cp38-win_amd64.whl
200
/google_drive/MyDrive/NVIDIA/torch-1.13.0+cu116-cp38-cp38-win_amd64.whl
File is already present


https://download.pytorch.org/whl/cu117/torch-1.13.0%2Bcu117-cp38-cp38-win_amd64.whl
200
/google_drive/MyDrive/NVIDIA/torch-1.13.0+cu117-cp38-cp38-win_amd64.whl
File is already present


https://download.pytorch.org/whl/cu116/torch-1.13.1%2Bcu116-cp38-cp38-win_amd64.whl
200
/google_drive/MyDrive/NVIDIA/torch-1.13.1+cu116-cp38-cp38-win_amd64.whl
File is already present


https://download.pytorch.org/whl/cu117/torch-1.13.1%2Bcu117-cp38-cp38-win_amd64.whl
200
/google_drive/MyDrive/NVIDIA/torch-1.13.1+cu117-cp38-cp38-win_amd64.whl
File is already present


https://developer.download.nvidia.com/compute/cuda/12.0.0/local_installers/cuda_12.0.0_527.41_windows.exe
200
/google_drive/MyDrive/NVIDIA/cuda_12.0.0_527.41_windows.exe
File is already present


